In [1]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT",""),
    api_key        = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version    = os.getenv("OPENAI_API_VERSION")
)

deployment_name    = os.getenv('DEPLOYMENT_NAME')

In [2]:
user_msg = """### Postgres SQL tables, with their properties:
#
# Employee(id, name, department_id)
# Department(id, name, address)
# Salary_Payments(id, employee_id, amount, date)
#
### Make a query to list the names of the departments which employed more than 10 employees in the last 3 months.
query:
"""

# user_msg = """### Postgres SQL tables, with their properties:
# #
# # Employee(id, name, department_id)
# # Department(id, name, address)
# # Salary_Payments(id, employee_id, amount, date)
# #
# ### 최근 3 개월간 직원을 10명 이상 뽑은 부서의 이름을 수집하는 쿼리를 만들어줘.
# query: 
# """

response = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": user_msg},
    ],
  temperature=0,
  max_tokens=500
)

print(response.choices[0].message.content)

To list the names of the departments which employed more than 10 employees in the last 3 months, you can use the following SQL query:

```sql
SELECT d.name
FROM Department d
JOIN Employee e ON d.id = e.department_id
JOIN Salary_Payments sp ON e.id = sp.employee_id
WHERE sp.date >= NOW() - INTERVAL '3 months'
GROUP BY d.name
HAVING COUNT(DISTINCT e.id) > 10;
```

### Explanation:
1. **JOIN Department and Employee**: This joins the `Department` and `Employee` tables on the `department_id` field.
2. **JOIN Salary_Payments**: This further joins the `Salary_Payments` table on the `employee_id` field.
3. **WHERE clause**: Filters the records to include only those salary payments made in the last 3 months.
4. **GROUP BY**: Groups the results by the department name.
5. **HAVING clause**: Filters the groups to include only those departments that have more than 10 distinct employees.

This query ensures that you get the names of departments that have employed more than 10 employees in the last 3

In [3]:
user_msg = """###Can you explain what does this code do? 
# Code: 
SELECT d.name
FROM Department d
WHERE d.id IN (
    SELECT e.department_id
    FROM Employee e
    WHERE e.id IN (
        SELECT sp.employee_id
        FROM Salary_Payments sp
        WHERE sp.date >= current_date - interval '3 months'
    )
    GROUP BY e.department_id
    HAVING COUNT(e.id) > 10
)
# Answer: 
"""

# user_msg = """###이 코드가 무엇을 수행하는지 설명해줘?
# #Code:
# SELECT d.name
# FROM Department d
# WHERE d.id IN (
#     SELECT e.department_id
#     FROM Employee e
#     WHERE e.id IN (
#         SELECT sp.employee_id
#         FROM Salary_Payments sp
#         WHERE sp.date >= current_date - interval '3 months'
#     )
#     GROUP BY e.department_id
#     HAVING COUNT(e.id) > 10
# )
# Answer:
# """
   
response = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": user_msg},
    ],
   max_tokens=250)

print(response.choices[0].message.content)

Certainly! Let's break down what this SQL query does step by step.

### Explanation

1. **Outer Query:**
   - The outer query selects the `name` of departments from the `Department` table (`d`).

   ```sql
   SELECT d.name
   FROM Department d
   ```

2. **WHERE Clause in the Outer Query:**
   - The outer query has a `WHERE` clause that checks if the `id` of the department is in a subquery.
   
   ```sql
   WHERE d.id IN (
   ```

3. **First Subquery:**
   - This subquery selects `department_id` from the `Employee` table (`e`). Hence, the outer query is interested in departments that satisfy the conditions inside this subquery.
   
   ```sql
   SELECT e.department_id
   FROM Employee e
   ```

4. **Second Subquery:**
   - Inside the first subquery, there is another subquery that is part of a `WHERE` clause. This subquery selects `employee_id` from the `Salary_Payments` table (`sp`) where the `date` is within the last 3 months from the current date.
   
   ```
